# Importing required libraries

In [1]:
import requests              # For making HTTP requests to fetch web pages
from bs4 import BeautifulSoup # For parsing HTML content
from csv import writer        # For writing scraped data into CSV file
from time import sleep        # To pause execution (simulate human-like scraping)
from random import choice     # To randomly select a quote

In [2]:
# List to store all scraped quotes
all_quotes = []

In [3]:
# Base URL of the website (constant part)
base_url = "http://quotes.toscrape.com/"

# Starting page (this will change as we move through pagination)
url = "/page/1"

In [4]:
page_num = 1
all_quotes = []

# Loop until there are no more pages left

while url:
    res = requests.get(f"{base_url}{url}")   # Make HTTP request
    print(f"Scraping page {page_num}...")    # Show current page
    
    # Parse the HTML content
    soup = BeautifulSoup(res.text, "html.parser")
    quotes = soup.find_all(class_="quote")
    
    # Extract all quote elements from the page
    for quote in quotes:
        all_quotes.append({
            "text": quote.find(class_="text").get_text(),
            "author": quote.find(class_="author").get_text(),
            "bio-link": quote.find("a")["href"]
        })

    # Find the "Next" button for pagination
    next_btn = soup.find(class_="next")
    url = next_btn.find("a")["href"] if next_btn else None
    
    page_num += 1
    sleep(1)   # Pause to avoid hitting server too fast

print("-"*50)
print(f"Scraping completed. Total pages scraped: {page_num-1}")
print(f"Total quotes scraped: {len(all_quotes)}")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
--------------------------------------------------
Scraping completed. Total pages scraped: 10
Total quotes scraped: 100


In [5]:
from random import choice

# At this point, all quotes have already been scraped into all_quotes list
print(f"Total quotes scraped: {len(all_quotes)}")

# Randomly select one quote from all scraped quotes
quote = choice(all_quotes)

# Set number of guesses allowed
remaining_guesses = 4

print("Here's a quote:  ")
print(quote["text"])   # Display the quote text

Total quotes scraped: 100
Here's a quote:  
“I have not failed. I've just found 10,000 ways that won't work.”


In [6]:
# Start the guessing game loop
guess = ''
while guess.lower() != quote["author"].lower() and remaining_guesses > 0:
    # Ask the user to guess the author
    guess = input(f"Who said this quote? Guesses remaining {remaining_guesses}: ")

    # If the guess is correct
    if guess.lower() == quote["author"].lower():
        print("CONGRATULATIONS!!! YOU GOT IT RIGHT")
        break

    # Reduce the number of guesses left
    remaining_guesses -= 1

    # Provide hints depending on how many guesses are left
    if remaining_guesses == 3:
        # Hint 1: Scrape author's bio page to get birth date and place
        res = requests.get(f"{base_url}{quote['bio-link']}")
        soup = BeautifulSoup(res.text, "html.parser")
        birth_date = soup.find(class_="author-born-date").get_text()
        birth_place = soup.find(class_="author-born-location").get_text()
        print(f"Hint: The author was born on {birth_date} {birth_place}")

    elif remaining_guesses == 2:
        # Hint 2: Show the first letter of the author's first name
        print(f"Hint: The author's first name starts with: {quote['author'][0]}")

    elif remaining_guesses == 1:
        # Hint 3: Show the first letter of the author's last name
        last_initial = quote["author"].split(" ")[1][0]
        print(f"Hint: The author's last name starts with: {last_initial}")

    else:
        # If no guesses left, reveal the correct answer
        print(f" Sorry, you ran out of guesses. The answer was {quote['author']}")

Hint: The author was born on February 11, 1847 in Milan, Ohio, The United States
Hint: The author's first name starts with: T
Hint: The author's last name starts with: A
 Sorry, you ran out of guesses. The answer was Thomas A. Edison


In [7]:
# Preview a few scraped quotes
all_quotes[:5]


[{'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein',
  'bio-link': '/author/Albert-Einstein'},
 {'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'author': 'J.K. Rowling',
  'bio-link': '/author/J-K-Rowling'},
 {'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein',
  'bio-link': '/author/Albert-Einstein'},
 {'text': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'author': 'Jane Austen',
  'bio-link': '/author/Jane-Austen'},
 {'text': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'author': 'Marilyn Monroe',
  'bio-link': '/author/Marilyn-Monroe'}]